# Парсинг

## Сбор текстов решений собраний акционеров

Для начала импортирются все необходимые для дальнейшей работы библиотеки

In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
import pandas as pd
import time
import numpy as np

По этому URL будет осуществляться поиск сообщений

In [42]:
url = "https://e-disclosure.ru/poisk-po-soobshheniyam"

Непосредственно название сообщения

In [43]:
name = 'Решения общих собраний участников (акционеров)'

Просто так собрать сообщения о раскрытии с e-disclosure нельзя. На сайте действуют достаточно хитрые AJAX скрипты, из-за которых динамичный сайт становится невозможно спарсить одним GET запросом. При поиске решений общих собраний по данным нам датам (3 квартал 2020 года) URL всей страницы не меняется и так и остается https://e-disclosure.ru/poisk-po-soobshheniyam. Поэтому было решено парсить ответ на POST запрос, имитируя через python requests действия AJAX скрипта. Для этого требовалось осуществить такой запрос самостоятельно по указанной ссылке, зайти в DevTools, выбрать Network, поставить фильтр на XHR запросы, а затем нажать кнопку поиск (с предварительно введенными данными поиска и выбранными датами). Далее убирался фильтр количества сообщений на страницу так, чтобы все 1200 сообщений были на одной странице (сообщений по запросу больше 1200, но сайт не позволяет отразить больше). После этого брался последний POST запрос, чей ответ и был нужным кодом таблицы результатов, копировалось содержимое его payload, чтобы имитировать аналогичный запрос

In [44]:
payload = 'lastPageSize: 2147483647@lastPageNumber: 1@query: @queryEvent: Решения общих собраний участников (акционеров)@eventTypeTerm: @radView: 0@dateStart: 01.07.2020@dateFinish: 30.09.2020@textfieldEvent: Решения общих собраний участников (акционеров)@radReg: FederalDistricts@districtsCheckboxGroup: -1@regionsCheckboxGroup: -1@branchClientLog: -1,1,14,21,3,2,19,6,8,15,7,16,22,18,12,10,17,20,13,11,4,5,9@branchesCheckboxGroup: -1@textfieldCompany: '

Сделаем из строки словарь. В данной строке пары key: value разделены символом @. Это не изначальный вариант запроса. Таким символом мы разделяем пары, чтобы точно не было повторов, когда будем по этим символам сплитить. Если взять, например символ =, то могут быть повторы и ошибки. А символ @ в этом запросе точно не встречается

In [45]:
pl = {}
for i in payload.split('@'):
    k, v = i.split(': ')
    pl[k] = v

Посылается POST запрос с параметрами из словаря

In [46]:
res = requests.post(url, params=pl)

Проверяется статус запроса (нас устроит ответ 200)

In [47]:
res

<Response [200]>

Преобразование сырого html кода в дерево BeautifulSoup

In [48]:
tree = BeautifulSoup(res.text, 'html')

Каждый элемент таблицы вывод содержится в теге < tr > ... </ tr >

In [50]:
table = tree.find_all('tr')

Последние два элемента списка отвечают за интерфейс сайта и не содержат карточки записей

In [51]:
table = table[:-2]

Должно получиться 1200 записей - это лимит для отображения по любому поисковому запросу

In [52]:
len(table)

1200

Каждому элементу списка ставится в соответствие словарь с:
- датой регистрации сообщения
- наименованием юридического лица
- страницей юридического лица на сайте e-disclosure
- названием сообщения
- ссылкой на страницу с сообщением

In [53]:
events = []
for item in tqdm(table):
    events.append({
        'registration_date': item.find_all('td')[0].text,
        'entity': item.find_all('td')[1].find_all('a')[0].text,
        'entity_page_link': item.find_all('td')[1].find_all('a')[0].get('href'),
        'event': item.find_all('td')[1].find_all('a')[1].text,
        'event_page_link': item.find_all('td')[1].find_all('a')[1].get('href')
    })

  0%|          | 0/1200 [00:00<?, ?it/s]

Словари преобразуются в датафрейм для удобства работы и последующего экспорта в Excel

In [57]:
data = pd.DataFrame(events)

In [58]:
data

,registration_date,entity,entity_page_link,event,event_page_link
0,30.09.2020 19:58,"ПАО ""СПБ Банк""",https://e-disclosure.ru/portal/company.aspx?id...,Решения общих собраний участников (акционеров),https://e-disclosure.ru/portal/event.aspx?Even...
1,30.09.2020 19:54,"ЗАО Агрофирма ""Нива""",https://e-disclosure.ru/portal/company.aspx?id...,Решения общих собраний участников (акционеров),https://e-disclosure.ru/portal/event.aspx?Even...
2,30.09.2020 19:46,"ОАО ""ДОРСТРОЙ""",https://e-disclosure.ru/portal/company.aspx?id...,Решения общих собраний участников (акционеров),https://e-disclosure.ru/portal/event.aspx?Even...
3,30.09.2020 19:24,ООО КСН «Структурные инвестиции 1»,https://e-disclosure.ru/portal/company.aspx?id...,Решения общих собраний участников (акционеров),https://e-disclosure.ru/portal/event.aspx?Even...
4,30.09.2020 19:12,"ОАО ""КАНАТ""",https://e-disclosure.ru/portal/company.aspx?id...,Решения общих собраний участников (акционеров),https://e-disclosure.ru/portal/event.aspx?Even...
...,...,...,...,...,...
1195,25.08.2020 08:07,ООО «Магистраль двух столиц»,https://e-disclosure.ru/portal/company.aspx?id...,Решения общих собраний участников (акционеров),https://e-disclosure.ru/portal/event.aspx?Even...
1196,25.08.2020 08:05,"ПАО ""СПБ Банк""",https://e-disclosure.ru/portal/company.aspx?id...,Решения совета директоров (наблюдательного сов...,https://e-disclosure.ru/portal/event.aspx?Even...
1197,24.08.2020 19:04,"ОАО ""ДОРСТРОЙ""",https://e-disclosure.ru/portal/company.aspx?id...,Решения совета директоров (наблюдательного сов...,https://e-disclosure.ru/portal/event.aspx?Even...
1198,24.08.2020 19:04,"ОАО ""ДОРСТРОЙ""",https://e-disclosure.ru/portal/company.aspx?id...,Созыв общего собрания участников (акционеров),https://e-disclosure.ru/portal/event.aspx?Even...


Фильтрация только решений общих собраний участников (акционеров)

In [59]:
df = data.copy()[data['event'] == name]

In [60]:
df

,registration_date,entity,entity_page_link,event,event_page_link
0,30.09.2020 19:58,"ПАО ""СПБ Банк""",https://e-disclosure.ru/portal/company.aspx?id...,Решения общих собраний участников (акционеров),https://e-disclosure.ru/portal/event.aspx?Even...
1,30.09.2020 19:54,"ЗАО Агрофирма ""Нива""",https://e-disclosure.ru/portal/company.aspx?id...,Решения общих собраний участников (акционеров),https://e-disclosure.ru/portal/event.aspx?Even...
2,30.09.2020 19:46,"ОАО ""ДОРСТРОЙ""",https://e-disclosure.ru/portal/company.aspx?id...,Решения общих собраний участников (акционеров),https://e-disclosure.ru/portal/event.aspx?Even...
3,30.09.2020 19:24,ООО КСН «Структурные инвестиции 1»,https://e-disclosure.ru/portal/company.aspx?id...,Решения общих собраний участников (акционеров),https://e-disclosure.ru/portal/event.aspx?Even...
4,30.09.2020 19:12,"ОАО ""КАНАТ""",https://e-disclosure.ru/portal/company.aspx?id...,Решения общих собраний участников (акционеров),https://e-disclosure.ru/portal/event.aspx?Even...
...,...,...,...,...,...
1184,25.08.2020 09:34,"ПАО ""ЧИФ Союзинвест""",https://e-disclosure.ru/portal/company.aspx?id...,Решения общих собраний участников (акционеров),https://e-disclosure.ru/portal/event.aspx?Even...
1185,25.08.2020 09:34,"ПАО ""ЧИФ Союзинвест""",https://e-disclosure.ru/portal/company.aspx?id...,Решения общих собраний участников (акционеров),https://e-disclosure.ru/portal/event.aspx?Even...
1191,25.08.2020 08:39,"АО ""ЛЗЭП""",https://e-disclosure.ru/portal/company.aspx?id...,Решения общих собраний участников (акционеров),https://e-disclosure.ru/portal/event.aspx?Even...
1195,25.08.2020 08:07,ООО «Магистраль двух столиц»,https://e-disclosure.ru/portal/company.aspx?id...,Решения общих собраний участников (акционеров),https://e-disclosure.ru/portal/event.aspx?Even...


Теперь для каждой записи датафрейма по ссылке сообщения необходимо достать текст сообщения, который находится в теге < div > с особым указанием на style. Занимает это порядка 20 минут

In [15]:
texts = []
for link in tqdm(df['event_page_link'].values):
    res = requests.get(link)
    bs = BeautifulSoup(res.text, 'html')
    text = bs.body.find_all('div', {'style': 'word-break: break-word; word-wrap: break-word; white-space: pre-wrap;'})[0].text
    if text == '':
        print(link)
    texts.append(text)
    time.sleep(1)

  0%|          | 0/531 [00:00<?, ?it/s]

Тексты добавляются в отдельную колонку для централизованной работы именно с таблицей, а не отдельными массивами данных

In [16]:
df['text'] = texts

## Rule-based выделение информации из текстов

Требуется достать из данных:
- Полное и сокращенное наименование эмитента
- Адрес, ИНН, ОГРН эмитента
- Дата и форма собрания
- Наименование и ИНН утвержденного аудитора + тип отчетности, которую ему поручено проверять (при наличии)
- Утвержденный состав совета директоров (при наличии)
- Поднимался ли на собрании вопрос о выплате дивидендов и если да, то какое решение принято (3 варианта: “принято решение выплатить дивиденды”, “принято решение не выплачивать дивиденды”, “вопрос не поднимался”)

Токенизация текстов - разбиение на отдельные куски для удобства анализа и поиска сущностей

In [24]:
texts = df['text'].apply(lambda x: x.split('\n'))

Пусты токены очевидно не информативны, от них избавляемся

In [25]:
texts = texts.apply(lambda t: [i for i in t if i != ''])

Функция для обработки даты. На вход - строка, содержащая дату, на выход - дата в с=едином стиле без лишних символов

In [26]:
def form_date(s):
    new = s.split('года')[0]
    new = new.split('г.')[0]
    new = new.replace('«', '')
    new = new.replace('»', '')
    new = new.replace('сентября', '.09.')
    new = new.replace('августа', '.08.')
    new = new.replace('июля', '.07.')
    new = new.replace(' ', '')
    return new

Функция обработки формы собрания. Если во входных данных содержатся ключевые слова, то это - совместное присутствие. В любом ином случае - заочное голосование. Эта функция стразует от отсутствия формы собрания: если в найденных токенах есть информация про присутствие, то выводим его, если нет, что презюмируем заочное голосование, так как иных вариантов нет

In [27]:
def form_form(f):
    if 'Собрание' in f or 'собрание' in f or 'присутствие' in f or 'совместное' in f:
        return 'совместное присутствие'
    else:
        return 'заочное голосование'

Функция для обработки списка Совета директоров. На вход получает токены с ФИО и символами, которые шли рядом. На выход - список в едином формате, но с лишними символами

In [28]:
def form_dir(lst):
    # пустой список - нет директоров 
    # (обрезание было достаточно широким, чтобы захватить хотя бы одно ФИО, но тут пусто)
    if len(lst) == 0:
        return 'вопрос не поднимался'
    # если всего одна строка, то в ней сразу все ФИО
    elif len(lst) == 1:
        s = lst[0]
        # сплит по знакам препинания
        if s.count(',') > 0 and s.count(';') > 0:
            new = []
            for i in s.split(','):
                if not s.count(';'):
                    new.append(i)
                else:
                    new += i.split(';')
            return new
        # часто ФИО через запятую идут
        elif s.count(',') > 0:
            return s.split(',')
        # реже через точку-запятую
        elif s.count(';') > 0:
            return s.split(';')
        # и снова часто списком
        elif s[0].isdigit():
            return [s]
        else:
            # новый список для особого случая
            new = []
            # индексы для движения окном по строке
            start_idx = 0
            finish_idx = 0
            for i in range(len(s) - 1):
                # если сразу после строчной без пробела заглавная - начался другой директор
                if s[i].islower() and s[i + 1].isupper():
                    finish_idx = i
                    new.append(s[start_idx:finish_idx + 1])
                    start_idx = i + 1
            if len(new) == 0:
                return 'вопрос не поднимался'
            return new
    else:
        # опять новый список, иначе если нет никакой информации, функция упадет с ошибкой Unbound
        new = []
        # одним циклом
        for i in range(len(lst)):
            # пустые строки не нужны
            if lst[i].strip() == '':
                continue
            #  строки со служебными символами тоже
            if lst[i].strip()[0] == '№':
                return []
            if '№' in lst[i]:
                continue
            # аудитор идет отдельно от директора, но утверждать аудитора может совет директоров
            if 'аудит' in lst[i].strip():
                return []
            # если первый и третий символы разные по типу, то исключаются пункты структуры (типа 1.2)
            if lst[i].strip()[0].isdigit() and not lst[i].strip()[2].isdigit():
                new.append(lst[i].strip())
                # на точке список кончается
                if lst[i].endswith('.'):
                    break
            # стразовка от выхода за границы списка при взятии следующего элемента
            try:
                # если меняется паттерн нумерации пунктов, то один список закончился и начался другой
                if lst[i].strip()[1] == '.' and lst[i + 1].strip()[1] != '.':
                    return new
                if lst[i].strip()[1] == '\t' and lst[i + 1].strip()[1] != '\t':
                    return new
                if lst[i].strip()[1] == ')' and lst[i + 1].strip()[1] != ')':
                    return new
                # если следующий пункт меньше предыдущего, то список кончился и продолжается другой
                if lst[i].strip()[0].isdigit() and lst[i + 1].strip()[0].isdigit():
                    if int(lst[i].strip()[0]) >= int(lst[i + 1].strip()[0]):
                        return new
            except Exception as e:
                break
        #  возвращаем получившийся список
        return new

Функция для унификации вида списков директоров. На вход - необработанные списки, на выход - одна строка со всеми фамилиями без лишних символов

In [29]:
def format_dir(s):
    # новый список для директоров
    new = []
    # если пришла строка а не список, то выходим
    if s == 'вопрос не поднимался':
        return s
    # за один проход по списку
    for i in range(len(s)):
        tmp = s[i]
        # бывают пустые строки
        if tmp == '':
            continue
        #  по тире справа директор
        if '-' in tmp:
            tmp = tmp[tmp.index('-') + 1:].strip()
        elif '–' in tmp:
            tmp = tmp[tmp.index('–') + 1:].strip()
        elif '—' in tmp:
            tmp = tmp[tmp.index('—') + 1:].strip()
        # если не сработало, то ищем по заглавным буквам
        f_upper = 0
        first_upper = 0
        for j in range(len(tmp)):
            if tmp[j].isupper():
                f_upper += 1
                if f_upper == 1:
                    # индекс первой заглавной сохраняем, это может быть начало фамилии
                    first_upper = j
        # больше трех заглавных - точно не ФИО, в нем 3 заглавных, но если больше 7
        # то вполне возможно идет список ФИО
        if f_upper > 3 and f_upper < 7:
            continue
        else:
            # если в конце буква, так и добавляем
            if tmp[-1].isalpha():
                new.append(tmp[first_upper:])
            else:
                # если нет, то обрезаем
                new.append(tmp[first_upper:-1])
    # вывод одной строкой, чтобы в таблице не выделялось потом
    return ', '.join(new)

Функция для поиска аудиторской организации. На вход - токены с информацией об утверждении аудиторской организации, на выход - название организации, ИНН (при наличии), тип проверяемой отчетности (при наличии)

In [30]:
def form_audit(s):
    # инициализация переменных для вывода
    comp = 0
    acc = 'не указан'
    comp_inn = 'не указан'
    # паттерн всегда одинаков, но не расположение слов относительно друг друга
    if 'Утвердить' in s and 'аудит' in s:
        # если они идут слитно
        if 'Утвердить аудитором' in s:
            # подстрока после этой фразы
            sub = s[s.index('Утвердить аудитором') + 19:]
            # дальше точки аудитора не будет
            if '.' in sub:
                sub = sub[:sub.index('.') + 1]
            # если точки не было, то можно по году грамматически смотреть
            # запятая - аудитор был раньше назван
            if 'год,' in sub:
                sub = sub[:sub.index('год,') - 4]
            # пробел - аудитора еще не было, был только 2020 год объявлен
            if 'год ' in sub:
                sub = sub[sub.index('год ') + 4: ]
            # после двоеточия он скорее всего и будет
            if 'год:' in sub:
                sub = sub[sub.index('год:') + 4: ]
            # в этой подстроке сейчас находится отчетность, она почти всегда рядом с аудитором
            if 'отчетн' in sub:
                #  но видов ее упоминается всего три
                if 'бухгал' in sub:
                    if 'финансово' in sub:
                        acc = 'бухгалтерская (финансовая) отчетность'
                    else:
                        acc = 'бухгалтерская отчетность'
                if 'консолидир' in sub:
                    acc = 'консолидированная отчетность'
            # сплиты по ниболее часто встречающимся началам названия аудитора
            # если потом есть точка, то название за нее не уйдет, если нет, то до конца строки
            if 'Общество' in sub:
                if '.' in sub:
                    comp = sub[sub.index('Общество'): sub.index('.')]
                else:
                    comp = sub[sub.index('Общество'):]
            elif 'Акционерное' in sub:
                if '(' in sub:
                    # редко бывает, что не до точки, а до открывающей скобки, где может быть ИНН
                    comp = sub[sub.index('Акционерное'): sub.index('(')]
                elif '.' in sub:
                    comp = sub[sub.index('Акционерное'): sub.index('.')]
                else:
                    comp = sub[sub.index('Акционерное'):]
            elif 'общество' in sub:
                if '.' in sub:
                    comp = sub[sub.index('общество'): sub.index('.')]
                elif '(' in sub:
                    comp = sub[sub.index('общество'): sub.index('(')]
                else:
                    comp = sub[sub.index('общество'):]
            elif 'Индивидуального' in sub:
                if '.' in sub:
                    comp = sub[sub.index('Индивидуального'): sub.index('.')]
                elif '(' in sub:
                    comp = sub[sub.index('Индивидуального'): sub.index('(')]
                else:
                    comp = sub[sub.index('Индивидуального'):]
            elif 'ООО' in sub:
                if '.' in sub:
                    comp = sub[sub.index('ООО'): sub.index('.')]
                else:
                    comp = sub[sub.index('ООО'):]
            elif 'Компания' in sub:
                if '.' in sub:
                    comp = sub[sub.index('Компания'): sub.index('.')]
                else:
                    comp = sub[sub.index('Компания'):]
            elif 'ПАО' in sub:
                if '.' in sub:
                    comp = sub[sub.index('ПАО'): sub.index('.')]
                else:
                    comp = sub[sub.index('ПАО'):]
            elif 'ЗАО' in sub:
                if '.' in sub:
                    comp = sub[sub.index('ЗАО'): sub.index('.')]
                else:
                    comp = sub[sub.index('ЗАО'):]
            elif 'ОАО' in sub:
                if '.' in sub:
                    comp = sub[sub.index('ОАО'): sub.index('.')]
                else:
                    comp = sub[sub.index('ОАО'):]
            elif 'АИКЦ' in sub:
                if '.' in sub:
                    comp = sub[sub.index('АИКЦ'): sub.index('.')]
                else:
                    comp = sub[sub.index('АИКЦ'):]
            elif 'ИП' in sub:
                if '.' in sub:
                    comp = sub[sub.index('ИП'): sub.index('.')]
                else:
                    comp = sub[sub.index('ИП'):]
            elif 'Аудиторскую' in sub:
                if '.' in sub:
                    comp = sub[sub.index('Аудиторскую'): sub.index('.')]
                else:
                    comp = sub[sub.index('Аудиторскую'):]
            # если аудитор все еще не найден, то сплит по знакам препинания, с особым вниманием к кавычкам
            elif '–' in sub and ('»' not in sub and sub.count('"') > 1):
                comp = sub.split('–')[-1].split('.')[0]
            elif '-' in sub and ('»' not in sub and sub.count('"') > 1):
                comp = sub.split('-')[-1].split('.')[0]
            elif '—' in sub and ('»' not in sub and sub.count('"') > 1):
                comp = sub.split('—')[-1].split('.')[0]
            # если нашли аудитора, то дальше обрабатываем, чтобы убрать лишние символы
            if isinstance(comp, str):
                # знаки препинания в конце
                if comp[-1] == ';':
                    comp = comp[:-1]
                elif ',' in comp:
                    comp = comp.split(',')[0]
                # в кобочках может быть ИНН, или просто рядом - справа обычно
                if 'ИНН' in comp:
                    strt = comp.index('ИНН')
                    fnsh = len(comp)
                    rng = fnsh - strt
                    comp_inn = []
                    for i in range(rng):
                        char = comp[strt + i]
                        # нашли первую цифру ИНН
                        if char.isdigit():
                            bg = strt + i
                            # циклом пока цифры не закончатся
                            while char.isdigit() and bg < fnsh:
                                comp_inn.append(char)
                                try:
                                    bg += 1
                                    char = comp[bg]
                                except Exception as e:
                                    break
                            break
                    comp_inn = ''.join(comp_inn)
                # с кавычками все сложно, надо по ним обрезать, если раньше не обрезалось
                if '»' in comp:
                    comp = comp[:comp.index('»') + 1]
                if '«' in comp and comp.count('"') == 1:
                    comp = comp[: comp.index('"') + 1]
                if comp.count('"') > 1:
                    comp = ' "'.join(comp.split('"')[:2]) + '"'

        else:
            # буквально в паре случаев аудитор назодится между этими словами, надо про это не забыть
            if 'Общество' in s[s.index('Утвердить '): s.index('аудит')]:
                comp = s[s.index('Утвердить '): s.index('аудит')]
        # у одной фирмы в названии 32 пробела, что выглядит не эстетично
        if isinstance(comp, str) and '                                ' in comp:
            comp = comp.replace('                                ', ' ')
    # возвращаем результаты
    return comp, comp_inn, acc

Функция для получения атрибута из токена сплитом по определенному слову. На вход - список токенов, номер текущего токена, его содержание, токен для сплита и дополнительный параметр для сплита по дефису (трем его вариантам)

In [31]:
def get_name(s, i, lst, txt='эмитента', allow_split=False):
    # если нужного токена нет, то сплита не будет
    if txt in s:
        # сам сплит
        name = s.split(txt)[-1]
        # если строка заканчивается, значит нужная сущность в следующей строке
        if name == '' or name == '.' or name == '(если применимо)' or name == ':':
            name = lst[i + 1].strip()
    else:
        # нет токена - ошибка, чтобы не мухлевать с фейковыми разбиениями
        raise ValueError()
    # иногда нужно разбивать по трем видам тире (все встречаются в текстах)
    if allow_split:
        if '–' in name:
            name = name.split('–')[-1]
        elif '-' in name:
            name = name.split('-')[-1]
        elif '—' in name:
            name = name.split('—')[-1]
    # дефолтные сплиты
    if ':' in name:
        name = name.split(':')[-1]
    elif '\t' in name:
        name = name.split('\t')[-1]
    else:
        # иногда вместо табуляции идут просто 4 пробела
        name = name.split('   ')[-1]
    # добавочный сплит по дефису
    if name.strip().startswith('-') or name.strip().startswith('–'):
        name = name.strip()[1:]
    # вывод без пробелов по сторонам
    return name.strip()

Функция для создания карточек сообщений. На вход - список токенов сообщения, далее отыскиваются наиболее подходящие под конкретные сущности или факты токены и инициализируются специальные функции из числа вышереализованных. На выход - словарь-карточка сообщения о раскрытии

In [32]:
def process(lst, tmp):
    # флаги сущностей, если флаг поднят - больше искать не нужно
    f_name, s_name = False, False
    ad, e_ogrn, e_inn = False, False, False
    dt, fm = False, False
    e_dir, s_dir, f_dir = False, False, False
    list_dir = []
    e_div, div_status = False, None
    e_audit, audit_status = False, None
    # поиск всей информации за один проход по массиву, иначе будет просадка в скорости
    for i in range(len(lst)):
        # пробелы слева и справа не нужны
        s = lst[i].strip()
        # в пункте 1.1 полное наименование
        if s.startswith('1.1') and not f_name:
            full = get_name(s, i, lst)
            f_name = True
        # в 1.2 - сокращенное
        if s.startswith('1.2') and not s_name:
            short = get_name(s, i, lst)
            s_name = True
        # в 1.3 - адрес
        if (s.startswith('1.3') or s.startswith('1 3')) and not ad:
            address = get_name(s, i, lst)
            ad = True
        # в 1.4 - ОГРН
        if s.startswith('1.4') and not e_ogrn:
            ogrn = get_name(s, i, lst)
            e_ogrn = True
        # в 1.5 - ИНН
        if s.startswith('1.5') and not e_inn:
            inn = get_name(s, i, lst)
            e_inn = True
        # пункт 1.8 может содержать информацию про дату собрания, а может и вовсе отсутствовать
        if s.startswith('1.8') and (not dt):
            # выделение нужного токена
            if 'сообщение' in s:
                date = get_name(s, i, lst, txt='сообщение')
                # эта фраза идет в конце строки, после нее даты точно нет, а вот в следующей строке - есть
                if date == '(если применимо)':
                    date = lst[i + 1]
                dt = True
        # если в 1.8 было пусто, то переход во вторучю часть и поиск по смыслу
        if (s.startswith('Дата ') or 'Дата ' in s) and ('проведения' in s or 'появления' in s) and not dt and s.count(':') < 2:
            # эвристика по входным данным
            if 'проведения' in s:
                date = get_name(s, i, lst, txt='собрания')
            elif 'появления' in s:
                date = get_name(s, i, lst, txt='события')
            dt = True
        # пункт 2.3 (при его наличии) также содержит дату собрания
        if s.startswith('2.3') and not dt and s.count(':') > 1:
            # а также больше одного символа ":"
            sub = s.replace(':', '@').replace('@', ':', 1)
            date = get_name(sub, i, lst)
            dt = True
        # аналогично, но для 1 ":"
        if s.strip().startswith('Дата,') and 'собрания' in s and not dt:
            sub = s.replace(':', '@').replace('@', ':', 1)
            date = get_name(sub, i, lst)
            dt = True
        # форма собрания может быть в одной строке, может быть в следующей после объявления, а может и через одну
        if ('Форма' in s or 'форма' in s) and 'проведения' in s and 'эмитента' in s and ':' in s and not fm:
            form = get_name(s, i, lst)
            fm = True
        # иногда форма отделяется специальными символами
        if ('Форма' in s or 'форма' in s) and 'проведения' in s and (':' in s or '–' in s or '-' in s) and not fm:
            form = get_name(s, i, lst, txt='проведения', allow_split=True)
            fm = True
        # или просто словами без каких либо грамматических разделов частей предложений
        if ('Форма' in s or 'форма' in s) and 'проведения' in s and not fm:
            if 'эмитента' in s:
                form = get_name(s, i, lst, allow_split=True)
            else:
                form = get_name(s, i, lst, txt='собрания', allow_split=True)
            fm = True
        # если ничего не помогло, значит форма идет прямо после слова "собрания" в нужном токене
        if ('Форма' in s or 'форма' in s) and 'собрания' in s and not fm:
            form = get_name(s, i, lst, txt='собрания')
            fm= True
        # поиск с учетом заглавных букв занимает больше места без изменения смылса, проще их убирать
        if 'збра' in s and 'овет' in s and 'директ' in s and not e_dir and 'аудит' not in s:
            e_dir = True
        # если в токене упоминается аудитор, то в нем нет списка совета директоров, 
        # кроме нескольки абсолютно не структурированных токенов, где текст идет вообще без разделов на части
        if e_dir and 'Избрать' in s and 'овет' in s and 'ирект' in s and not f_dir and 'аудит' not in s:
            # если найден список ФИО, то его нужно забрать либо по ":"
            if ':' in s:
                if len(s.split(':')[-1]) > 2:
                    list_dir.append(s.split(':')[-1])
                # либо по перечислению
                elif lst[i + 1].strip()[0].isdigit():
                    s_dir = True
                    idx = 1
                    while s_dir:
                        # список пополняется до "."
                        list_dir.append(lst[i + idx])
                        if lst[i + idx].endswith('.'):
                            s_dir = False
                            f_dir = True
                        idx += 1
            else:
                # если нет вообще никаких указателей, то берется вся нужная часть токена для пост-обработки
                list_dir.append(s.split('директоров')[-1].strip())
        # есть вариант разделения вообще без знаков препинания, 
        # тут только по слитной заглавной букве можно ограничить список
        if e_dir and 'Избраны' in s and 'овет' in s and 'ирект' in s and not f_dir and 'аудит' not in s:
            list_dir.append(s.split('директоров')[-1].strip())
            list_dir[0] = list_dir[0].split('По')[0].split(':')[-1]
        # если дивиденды упоминаются, значит их будут делить
        if not e_div and 'ыплат' in s and 'дивиден' in s:
            e_div = True
        # если речь про дивиденды уже заходила
        if e_div:
            # то надо искать выплаты или не выплаты, в больгинстве случаев все прямолинейно
            if 'ивиден' in s and ('не выпла' in s or 'не пла' in s or 'не утв' in s or 'не распр' in s or 'убыт' in s):
                div_status = 'принято решение не выплачивать дивиденды'
            # но не всегда, тогда можно смотреть на расстояние между дивидендами и отрицанием
            elif 'ивиден' in s and 'не' in s:
                # 30 символов вполне разменая черта (в среднем, слово ~ 7-8 символов)
                if (s.index('ивиден') < s.index('не') + 30):
                    div_status = 'принято решение выплатить дивиденды'
        # про аудиторов везде написано одинаково
        if ('Утв' in s or 'утв' in s) and ('аудит' in s or 'Аудит' in s):
            e_audit = True
        # если утвердили, значит нужно искать организацию
        if e_audit:
            if 'Утв' in s and 'аудит' in s:
                audit_status = form_audit(s)
    # если про аудиторов ничего не известно, то вопрос не поднимался, раз даже упоминаний нет
    # если же аудитора нашли, то можно попробовать забрать ИНН и тип отчетности, они рядом находятся
    # просто в рандомной последовательности
    if e_audit and audit_status != None:
        auditor, aud_inn, aud_acc = audit_status
        if auditor == 0 or auditor == '0':
            auditor, aud_inn, aud_acc = 'вопрос не поднимался', 'вопрос не поднимался', 'вопрос не поднимался'
    elif not e_audit:
        auditor, aud_inn, aud_acc = 'вопрос не поднимался', 'вопрос не поднимался', 'вопрос не поднимался'
    else:
        auditor, aud_inn, aud_acc = 'вопрос не поднимался', 'вопрос не поднимался', 'вопрос не поднимался'
    # если дивиденды не упоминалсиь, то и решения про них не было
    if e_div and div_status is None:
        div_status = 'принято решение выплатить дивиденды'
    elif not e_div:
        div_status = 'вопрос не поднимался'
    # если не удалось найти специальные токены присутствия, то вариант всего один - заочное голосование
    if not fm:
        form = 'заочное голосование'
    # форматирование даты
    date = form_date(date)
    # форматирование токена с формой собрания
    form = form_form(form)
    # форматирование списка директоров
    list_dir = form_dir(list_dir)
    # если совет директоров упоминается и список не пустой (а есть сообщения, где Совет есть, а списка нет),
    # то обрабатываем его и приводим к строке
    if e_dir and len(list_dir) > 0:
        director = format_dir(list_dir)
    else:
        director = 'вопрос не поднимался'
    # помещаем все в карточку и выводим ее
    return {
        'полное наименование': full,
        'сокращенное наименование': short,
        'адрес': address,
        'ИНН': inn,
        'ОГРН': ogrn,
        'дата собрания': date,
        'форма собрания': form,
        'наименование аудитора': auditor,
        'ИНН аудитора': aud_inn,
        'тип отчетности': aud_acc,
        'состав совета директоров': director,
        'выплата дивидендов': div_status
    }

Применение главной функции ко всем сообщениям с сохранением карточек в списке

In [33]:
arr = []
for i in tqdm(range(texts.values.shape[0])):
    addr = process(texts.values[i], i)
    arr.append(addr)

  0%|          | 0/531 [00:00<?, ?it/s]

Сводная таблица получившихся карточек

In [34]:
new_df = pd.DataFrame(arr)

Объединение с изначальной таблицей сообщений

In [35]:
res_df = pd.concat([df.reset_index(drop=True), new_df], axis=1)

Удаление ненужных столбцов

In [36]:
res_df = res_df.drop(columns=['entity', 'entity_page_link', 'event', 'event_page_link'])

Унификация наименований столбцов

In [37]:
res_df = res_df.rename(columns={'registration_date': 'дата регистрации сообщения', 'text': 'текст сообщения'})

Вывод первых пяти строк-сообщений

In [67]:
res_df.head()

,дата регистрации сообщения,текст сообщения,полное наименование,сокращенное наименование,адрес,ИНН,ОГРН,дата собрания,форма собрания,наименование аудитора,ИНН аудитора,тип отчетности,состав совета директоров,выплата дивидендов
0,30.09.2020 19:58,Сообщение о существенном факте о проведении об...,Публичное акционерное общество «Бест Эффортс Б...,ПАО «Бест Эффортс Банк»,"Российская Федерация, город Москва",7831000034,1037700041323,30.09.2010,заочное голосование,Утвердить Общество с ограниченной ответственно...,не указан,не указан,"Жизненко Олег Михайлович, Бурдонова Марина Пав...",принято решение не выплачивать дивиденды
1,30.09.2020 19:54,Решения общих собраний участников (акционеров)...,"Закрытое акционерное общество Агрофирма ""Нива""","ЗАО Агрофирма ""Нива""","140090, Московская область, г. Дзержинский, ул...",5027028404,1035010951722,30.09.2020,заочное голосование,Общество с ограниченной ответственностью «Конс...,не указан,не указан,"Данкверт Юлия Сергеевна, Мишин Роман Александр...",принято решение выплатить дивиденды
2,30.09.2020 19:46,Решения общих собраний участников (акционеров)...,Открытое акционерное общество по строительству...,"ОАО ""ДОРСТРОЙ""","347800, Ростовская область, г. Каменск-Шахтинс...",6147002495,1026102107008,30.09.2020,совместное присутствие,Общество с ограниченной ответственностью «Аудит»,6147005739,не указан,"Имедашвили Николая Гивиевича, Кирсанова Владим...",принято решение не выплачивать дивиденды
3,30.09.2020 19:24,Решения общих собраний участников (акционеров)...,Общество с ограниченной ответственностью «Комп...,ООО КСН «Структурные инвестиции 1»,"125171, Российская Федерация, г. Москва, Ленин...",7743928024,1147746610725,30.09.2020,совместное присутствие,вопрос не поднимался,вопрос не поднимался,вопрос не поднимался,вопрос не поднимался,вопрос не поднимался
4,30.09.2020 19:12,Решения общих собраний участников (акционеров)...,"Открытое акционерное общество ""КАНАТ""","ОАО ""КАНАТ""","Россия, Санкт-Петербург, 197110, Петровский пр...",7813054069,1027806857693,28.09.2020,заочное голосование,ООО «Агентство «Бизнес-Проект»,не указан,не указан,вопрос не поднимался,вопрос не поднимался


Сохранение результата в Excel-таблице

In [39]:
res_df.to_excel('Сообщения о раскрытии.xlsx', index=False)

Сохранение результата в CSV-таблице

In [40]:
res_df.to_csv('Сообщения о раскрытии.csv', index=False)

## Итог

Rule-based подход, несмотря на свою кажущуюся простоту, содержит в себе ряд сложностей - получаемые "правила"-эвристики не инвариантны к изменениям входных данных. Но даже сам поиск отдельных правил может представлять из себя весьма не тривиальную задачу, сводящуюся к поиску кажущегося наиболее приемлемым решения, а затем дополнения его десятками строк исклчюений.

Отдельно стоит отметить крайне слабую структурированность входных данных. Несмотря на кажущуюся упорядоченность по отдельным пунктам, абзацам и т.д., это вовсе не так. Из 518 сообщений всегда находилось сообщение, чье форматирование не вписывается в выведенные правила, что заставляло искать новые решения, при этом не ломая уже достигнутые. Отдельные блоки информации уже после пункта 1.5 сообщения перестают быть детектируемыми по номерам или структурным показателям. Единственным способом становится вывод максимально универсальных эвристик для поиска семантических совпадений, выявления их грамматического и пунктуационного оформления и формализации в виде нового правила.

Многие сообщения почти не содержат полезной для данного задания информации. Хотя гораздо сложнее было с сообщениями, в которых нужная информация либо отсутствует (совет директоров выбирался, но результатов мы вам не скажем), либо оформлена макисмально сбивающе (взятие названия юридического лица в разные кавечки: ООО «Рога и Копыта", например). Это был сложный, но интересный опыт.

Было реализовано очень много костылей. Возможно, часть из них можно убрать, используя регулярные выражения и циклы, чтобы не было постоянных повторов. Однако неоспоримый плюс данной реализации - скорость ее работы. 531 сообщение обрабатывается почти мгновенно. Если использовать более "тяжелые" методы, то скорость заметно просядет. Так, можно было бы еще применять лингвистические модели и библиотеки, например nltk, для POS разметки и упрощения некоторых операций. 